Adding Text Data to Vector DB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q pinecone-client langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade --quiet  langchain-pinecone langchain-openai

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
import os
from langchain.text_splitter import TokenTextSplitter
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = 'OPEN AI KEY'
os.environ["PINECONE_API_KEY"] = 'PINECONE API KEY'
os.environ["PINECONE_INDEX_NAME"] = 'PINE CONE INDEX'

"""
#txt file location
text = '/content/drive/MyDrive/IBM_Spring24/paper scraping/mangrove_policy_text.txt'

# Function to split text from a file into chunks by every 6 sentences
def split_into_chunks_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return split_into_chunks(text)

def split_into_chunks(text):
    text_splitter = TokenTextSplitter(chunk_size=768, chunk_overlap=20)
    return text_splitter.split_text(text)

chunks = split_into_chunks_from_file(text)
"""

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from langchain_pinecone import Pinecone
embeddings = OpenAIEmbeddings()
db = Pinecone.from_existing_index(index_name="mangrove-index", embedding = embeddings)
#db.add_texts([chunk for chunk in chunks])

Setting up RAG + Runnign QnA

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
retriever = db.as_retriever()

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)
chain = ConversationalRetrievalChain.from_llm(llm, retriever= retriever, memory= memory)

In [ ]:
link = 'https://www.mangrovealliance.org/wp-content/uploads/2023/12/GMA-Policy-Brief_V6.pdf'

query_template = f"""
                  For the following query, provide detailed informational input regarding mangroves. If possible,
                  cite specific numbers/events/policies. If answer is not known, respond
                  'I'm not sure, but some relevant information can be found here: {link}'.

                  example:
                  Questions: Tell me about mangrove change in Senegal.
                  Answer: Mangroves in Senegal have undergone a notable decline since 1980, with the mangrove area in the country shrinking from 2300 km in 1990 to 1760 km in 2008. Over 30 years, the Saloum reserve of the biosphere's mangrove has lost approximately 40% of its area, amounting to about 750 km. Similarly, in Casamance, around 670 km of the ecosystem vanished during the same period. Despite these losses, recent studies indicate a potential recovery in certain areas, such as the Senegal-Gambia border of Saloum-Niumi and the Blis-Karone islands of Casamance. In these areas, women engaged in shellfish harvesting have observed changes in the mangrove forest, noting a decrease in density and a decline in the size and abundance of oysters and arches. The reasons behind the decline of mangroves in Senegal are multifaceted, including factors such as drought, the introduction of new techniques by both public and private actors, and the impact of the prolonged conflict in Casamance. During reforestation campaigns, women interviewed have voiced concerns about the degradation of mangroves and emphasized the importance of adopting sustainable management practices to address these issues.

                  The query is as follows:
"""

In [ ]:
query_input = 'tell me about important policies in Senegal for mangroves'
chain.run({'question': query_template + query_input})

'Some important policies in Senegal for protecting mangroves include the promotion of responsible and fair harvesting practices, the establishment of local "conventions" by local people with the support of NGOs and international organizations, and the implementation of the "Plant your tree" program. Additionally, there is a Mangrove Charter and national action plans in place to promote inclusive and long-term governance of mangroves. The country also participates in the West Africa Marine and Coastal Conservation Platform (PRCM) to coordinate mangrove conservation efforts at a sub-regional level.'

Generating Visualizations **NOT WORKING**

In [ ]:
!pip install -q tabulate langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import pandas as pd
import matplotlib.pyplot as plt
import json

def create_agent(filename: str):
    """
    Create an agent that can access and use a large language model (LLM).

    Args:
        filename: The path to the CSV file that contains the data.

    Returns:
        An agent that can access and use the LLM.
    """

    # Read the CSV file into a Pandas DataFrame.
    df = pd.read_csv(filename)

    # Create a Pandas DataFrame agent.
    return create_pandas_dataframe_agent(llm, df, verbose=False)

def query_agent(agent, query):
    """
    Query an agent and return the response as a string.

    Args:
        agent: The agent to query.
        query: The query to ask the agent.

    Returns:
        The response from the agent as a string.
    """

    prompt = (
        """
            For the following query, if it requires drawing a table, reply as follows:
            {"table": {"columns": ["column1", "column2", ...], "data": [[value1, value2, ...], [value1, value2, ...], ...]}}

            If the query requires creating a bar chart, reply as follows:
            {"bar": {"columns": ["A", "B", "C", ...], "data": [25, 24, 10, ...]}}

            If the query requires creating a line chart, reply as follows:
            {"line": {"columns": ["A", "B", "C", ...], "data": [25, 24, 10, ...]}}

            There can only be two types of chart, "bar" and "line".

            If it is just asking a question that requires neither, reply as follows:
            {"answer": "answer"}
            Example:
            {"answer": "The country with the greatest increase in mangrove area is 'Indonesia'"}

            If you do not know the answer, reply as follows:
            {"answer": "I do not know."}

            Return all output as a string.

            All strings in "columns" list and data list, should be in double quotes,

            For example: {"columns": ["country", "quantity"], "data": [["India", 1361], ["Brazil", 5164]]}

            Lets think step by step.

            Below is the query for the mangrove square footage dataset.
            Query:
            """
        + query
    )

    # Run the prompt through the agent.
    response = agent.run(prompt)

    # Convert the response to a string.
    return response.__str__()

def decode_response(response: str) -> dict:
    """This function converts the string response from the model to a dictionary object.

    Args:
        response (str): response from the model

    Returns:
        dict: dictionary with response data
    """
    if not response:
        raise ValueError("error processing response")
    return json.loads(response)

def write_response(response_dict: dict):
    """
    Write a response from agent.

    Args:
        response_dict: The response from the agent.

    Returns:
        None.
    """
    import matplotlib.pyplot as plt

    # Check if the response is an answer.
    if "answer" in response_dict:
        print(response_dict["answer"])

    # Check if the response is a bar chart.
    if "bar" in response_dict:
        data = response_dict["bar"]
        df = pd.DataFrame(data)
        df.set_index("columns", inplace=True)
        plt.figure(figsize=(10, 6))
        plt.bar(df.index, df.values)
        plt.xlabel('Columns')
        plt.ylabel('Values')
        plt.title('Bar Chart')
        plt.show()  # Show the bar chart inline

    # Check if the response is a line chart.
    if "line" in response_dict:
        data = response_dict["line"]
        df = pd.DataFrame(data)
        df.set_index("columns", inplace=True)
        plt.figure(figsize=(10, 6))
        for column in df.columns:
            plt.plot(df.index, df[column], label=column)
        plt.xlabel('Columns')
        plt.ylabel('Values')
        plt.title('Line Chart')
        plt.legend()
        plt.show()  # Show the line chart inline

    # Check if the response is a table.
    if "table" in response_dict:
        data = response_dict["table"]
        df = pd.DataFrame(data)
        print(df)


In [ ]:
#running the agent
data = '/content/drive/MyDrive/IBM_Spring24/Factor Research/mangrove_coverage.csv'

agent = create_agent(data)

In [ ]:
def run_tabular_rag(query):
  # Query the agent.
  response = query_agent(agent=agent, query=query)

  # Decode the response.
  decoded_response = decode_response(response)

  # Write the response to the Streamlit app.
  write_response(decoded_response)

In [ ]:
run_tabular_rag('make a line graph for Brazil')

ValueError: All arrays must be of the same length